In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive

In [ ]:
!pip install focal-loss  -q gwpy

In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from focal_loss import SparseCategoricalFocalLoss
# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
img_size = (224,224)

In [ ]:
def get_img_array(img_path, size, channels=1):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, color_mode="grayscale", target_size=size)

    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    if channels==3:
      array = np.stack([array, array, array], axis =-2)
      array = np.squeeze(array)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

In [ ]:
def make_gradcam_heatmap(img_array, grad_model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=1):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))

# GradCam EffNet

In [ ]:
  # supernet.trainable = False
from keras.layers import Dense,Flatten,GlobalAveragePooling2D, MaxPooling2D, BatchNormalization,Concatenate, Resizing
from keras import regularizers
def get_model():
    inputs = tf.keras.layers.Input((224,224,3))
    supernet = tf.keras.applications.EfficientNetV2B3(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    input_tensor = inputs
)

    count = 1
    for layer in supernet.layers:
      if count < 80:
          layer.trainable = False
      else:
          layer.trainable = True
      count = count + 1



    y2 = supernet(inputs)

    y2 = tf.keras.layers.GlobalAveragePooling2D()(y2)
    y2 = tf.keras.layers.Dense(256, activation='relu', kernel_initializer = tf.keras.initializers.HeUniform(4224))(y2)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer = tf.keras.initializers.GlorotUniform(4224))(y2)

    # Connect input and output through the Model class
    effNet = tf.keras.Model(inputs=inputs, outputs=outputs, name='model')

   # Compile the model
    effNet.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics='accuracy')

    return effNet
  

## Gradcam T

In [ ]:
effNet = get_model()

In [ ]:
effNet.load_weights("/gdrive/MyDrive/tuberculosis-pneumonia-classification/model_exp3_T2.h5")

In [ ]:
effNet.summary()

In [ ]:
img_path_T = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data/train_all_no_duplicates/P16072_1.png'

In [ ]:
last_conv_layer_name = "top-activation"

In [ ]:
print(effNet.layers[-1].activation)

In [ ]:
img_array = get_img_array(img_path_T, img_size, channels=3)

# Print what the top predicted class is
preds = effNet.predict(img_array)
print(preds)

# Remove last layer's softmax
effNet.layers[-1].activation = None
effNet.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics='accuracy')

# Print what the top predicted class is
preds = effNet.predict(img_array)
print(preds)

gradModel = tf.keras.Model(
        inputs=effNet.input,
        outputs=[effNet.get_layer("global_average_pooling2d_21").input,
                 effNet.output])


# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, gradModel, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
CAM_T_T_path1 = '/gdrive/MyDrive/AAIB/report_media/CAM_T_T21.jpg'

In [ ]:
save_and_display_gradcam(img_path_T, heatmap, CAM_T_T_path1)

## GradCam N

In [ ]:
img_path_N = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data/train_all_no_duplicates/P16043_1.png'

In [ ]:
effNet = get_model()

In [ ]:
effNet.load_weights("/gdrive/MyDrive/tuberculosis-pneumonia-classification/pablo/models/model_exp3_N2.h5")

In [ ]:
effNet.summary()

In [ ]:
last_conv_layer_name = "top-activation"

In [ ]:
print(effNet.layers[-1].activation)

In [ ]:
img_array = get_img_array(img_path_N, img_size, channels=3)

# Print what the top predicted class is
preds = effNet.predict(img_array)
print(preds)

# Remove last layer's softmax
effNet.layers[-1].activation = None
effNet.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics='accuracy')

# Print what the top predicted class is
preds = effNet.predict(img_array)
print(preds)

gradModel = tf.keras.Model(
        inputs=effNet.input,
        outputs=[effNet.get_layer("global_average_pooling2d_23").input,
                 effNet.output])


# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, gradModel, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
CAM_S_S_path = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data_for_report/CAM_S_S2.jpg'

In [ ]:
save_and_display_gradcam(img_path_N, heatmap, CAM_S_S_path)

## GradCam P

In [ ]:
img_path_P = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data/train_all_no_duplicates/P15529_1.jpeg'

In [ ]:
effNet = get_model()

In [ ]:
effNet.load_weights("/gdrive/MyDrive/tuberculosis-pneumonia-classification/model_exp3_P2.h5")

In [ ]:
effNet.summary()

In [ ]:
last_conv_layer_name = "top-activation"

In [ ]:
print(effNet.layers[-1].activation)

In [ ]:
img_array = get_img_array(img_path_P, img_size, channels=3)

# Print what the top predicted class is
preds = effNet.predict(img_array)
print(preds)

# Remove last layer's softmax
effNet.layers[-1].activation = None
effNet.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4), metrics='accuracy')

# Print what the top predicted class is
preds = effNet.predict(img_array)
print(preds)

gradModel = tf.keras.Model(
        inputs=effNet.input,
        outputs=[effNet.get_layer("global_average_pooling2d_28").input,
                 effNet.output])


# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, gradModel, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
CAM_P_P_path = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data_for_report/CAM_P_P21.jpg'

In [ ]:
save_and_display_gradcam(img_path_P, heatmap, CAM_P_P_path)

# GradCam ResNet

In [ ]:
model = keras.models.load_model('/gdrive/MyDrive/bestResNetL.h5')

In [ ]:
model.summary()

In [ ]:
last_conv_layer_name = "activation_15"

In [ ]:
img_path = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data/train_all_no_duplicates/P16072_1.png'

In [ ]:
img_array = get_img_array(img_path, img_size)

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print(preds)
grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, grad_model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))

In [ ]:
CAM_T_T_path1 = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/CAM_T_T21.jpg'

In [ ]:
save_and_display_gradcam(img_path, heatmap, CAM_T_T_path1)

In [ ]:
img_path_N = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data/train_all_no_duplicates/P16043_1.png'

In [ ]:
img_array = get_img_array(img_path_N, img_size)

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)

grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, grad_model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
CAM_S_S_path = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/CAM_S_S2.jpg'

In [ ]:
save_and_display_gradcam(img_path_N, heatmap, CAM_S_S_path)

In [ ]:
img_path_P = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/data/train_all_no_duplicates/P15529_1.jpeg'

In [ ]:
img_array = get_img_array(img_path_P, img_size)

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print(preds)

grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, grad_model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
CAM_P_P_path = '/gdrive/MyDrive/tuberculosis-pneumonia-classification/CAM_P_P21.jpg'

In [ ]:
save_and_display_gradcam(img_path_P, heatmap, CAM_P_P_path)